https://blog.myyellowroad.com/using-categorical-data-in-machine-learning-with-python-from-dummy-variables-to-deep-category-66041f734512

In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('/home/kai/data/shiyi/data/avazu-ctr-prediction/train', nrows=100000)

In [4]:
import numpy as np
from sklearn.metrics import log_loss
msk = np.random.rand(len(data)) < 0.8
features = [3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23]
X_train = data[msk].iloc[:,features]
X_test = data[~msk].iloc[:,features]
y_train = data[msk].iloc[:,1]
y_test = data[~msk].iloc[:,1]
print(log_loss(y_test,np.ones(len(y_test))*y_train.mean()))

0.46160842572806454


# label encoding

In [5]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
X_train_ordinal = X_train.values
X_test_ordinal = X_test.values
les = []
l = LogisticRegression()
r = RandomForestClassifier(n_estimators=25,max_depth=10)
for i in range(X_train_ordinal.shape[1]):
    le = preprocessing.LabelEncoder()
    le.fit(data.iloc[:,features].iloc[:, i])
    les.append(le)
    X_train_ordinal[:, i] = le.transform(X_train_ordinal[:, i])
    X_test_ordinal[:, i] = le.transform(X_test_ordinal[:, i])

In [5]:
l.fit(X_train_ordinal,y_train)
y_pred = l.predict_proba(X_test_ordinal)
print(log_loss(y_test,y_pred))

0.4344061257450101


In [6]:
r.fit(X_train_ordinal,y_train)
y_pred = r.predict_proba(X_test_ordinal)
print(log_loss(y_test,y_pred))

0.4114955094510522


# One Hot Encoding

In [10]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_train_ordinal)
X_train_one_hot = enc.transform(X_train_ordinal)
X_test_one_hot = enc.transform(X_test_ordinal)
l.fit(X_train_one_hot,y_train)
y_pred = l.predict_proba(X_test_one_hot)
print(log_loss(y_test,y_pred))
r.fit(X_train_one_hot,y_train)
y_pred = r.predict_proba(X_test_one_hot)
print(log_loss(y_test,y_pred))
print(X_train_one_hot.shape)

0.40754774134678073
0.43215400119144154
(80069, 11472)


In [11]:
import copy
X_train_rare = copy.copy(X_train)
X_test_rare = copy.copy(X_test)
X_train_rare["test"]=0
X_test_rare["test"]=1
temp_df = pd.concat([X_train_rare,X_test_rare],axis=0)
names = list(X_train_rare.columns.values)
temp_df = pd.concat([X_train_rare,X_test_rare],axis=0)
for i in names:
    temp_df.loc[temp_df[i].value_counts()[temp_df[i]].values < 20, i] = "RARE_VALUE"
for i in range(temp_df.shape[1]):
    temp_df.iloc[:,i]=temp_df.iloc[:,i].astype('str')
X_train_rare = temp_df[temp_df["test"]=="0"].iloc[:,:-1].values
X_test_rare = temp_df[temp_df["test"]=="1"].iloc[:,:-1].values
for i in range(X_train_rare.shape[1]):
    le = preprocessing.LabelEncoder()
    le.fit(temp_df.iloc[:,:-1].iloc[:, i])
    les.append(le)
    X_train_rare[:, i] = le.transform(X_train_rare[:, i])
    X_test_rare[:, i] = le.transform(X_test_rare[:, i])
enc.fit(X_train_rare)
X_train_rare = enc.transform(X_train_rare)
X_test_rare = enc.transform(X_test_rare)

In [12]:
l.fit(X_train_rare,y_train)
y_pred = l.predict_proba(X_test_rare)
print(log_loss(y_test,y_pred))
r.fit(X_train_rare,y_train)
y_pred = r.predict_proba(X_test_rare)
print(log_loss(y_test,y_pred))
print(X_train_rare.shape)

0.4088065263503137
0.42305294188782705
(80069, 1554)


# Feature Hashing

In [13]:
X_train.head(2)

,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,44956a24,1,2,15706,320,50,1722,0,35,-1,79
1,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,711ee120,1,0,15704,320,50,1722,0,35,100084,79


In [22]:
X_train.shape

(80069, 20)

In [24]:
from sklearn.feature_extraction import FeatureHasher
X_train_hash = copy.copy(X_train)
X_test_hash = copy.copy(X_test)
for i in range(X_train_hash.shape[1]):
    X_train_hash.iloc[:,i]=X_train_hash.iloc[:,i].astype('str')
for i in range(X_test_hash.shape[1]):
    X_test_hash.iloc[:,i]=X_test_hash.iloc[:,i].astype('str')
h = FeatureHasher(n_features=100,input_type="string")
X_train_hash = h.transform(X_train_hash.values)
X_test_hash = h.transform(X_test_hash.values)

In [25]:
type(X_train_hash)

scipy.sparse.csr.csr_matrix

In [26]:
X_train_hash.shape

(80069, 100)

In [27]:
df = pd.DataFrame(X_test_hash.toarray())

In [28]:
df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
l.fit(X_train_hash,y_train)
y_pred = l.predict_proba(X_test_hash)
print(log_loss(y_test,y_pred))#0.4
r.fit(X_train_hash,y_train)
y_pred = r.predict_proba(X_test_hash)
print(log_loss(y_test,y_pred))

0.4212695876384164
0.4161355973932612


# Encoding categories with dataset statistics

In [41]:
X_train_count = copy.copy(X_train)
X_test_count = copy.copy(X_test)
X_train_count["test"]=0
X_test_count["test"]=1
temp_df = pd.concat([X_train_count,X_test_count],axis=0)
for i in range(temp_df.shape[1]):
    temp_df.iloc[:,i]=temp_df.iloc[:,i].astype('category')
X_train_count=temp_df[temp_df["test"]==0].iloc[:,:-1]
X_test_count=temp_df[temp_df["test"]==1].iloc[:,:-1]
for i in range(X_train_count.shape[1]):
    counts = X_train_count.iloc[:,i].value_counts()
    counts = counts.sort_index()
    counts = counts.fillna(0)
    counts += np.random.rand(len(counts))/1000 # add small random number so two different categorical values with
                                                # same counts will result in different but very close encoding values
    X_train_count.iloc[:,i].cat.categories = counts
    X_test_count.iloc[:,i].cat.categories = counts

In [42]:
l.fit(X_train_count,y_train)
y_pred = l.predict_proba(X_test_count)
print(log_loss(y_test,y_pred))

0.4339115045397575


In [44]:
r.fit(X_train_count,y_train)
y_pred = r.predict_proba(X_test_count)
print(log_loss(y_test,y_pred))

0.41006749357640637


In [47]:
X_train_ctr = copy.copy(X_train)
X_test_ctr = copy.copy(X_test)
X_train_ctr["test"]=0
X_test_ctr["test"]=1
temp_df = pd.concat([X_train_ctr,X_test_ctr],axis=0)
for i in range(temp_df.shape[1]):
    temp_df.iloc[:,i]=temp_df.iloc[:,i].astype('category')
X_train_ctr=temp_df[temp_df["test"]==0].iloc[:,:-1]
X_test_ctr=temp_df[temp_df["test"]==1].iloc[:,:-1]
temp_df = pd.concat([X_train_ctr,y_train],axis=1)
names = list(X_train_ctr.columns.values)
for i in names:
    means = temp_df.groupby(i)['click'].mean()
    means = means.fillna(sum(temp_df['click'])/len(temp_df['click'])) # fill na with overall mean
    means += np.random.rand(len(means))/1000
    X_train_ctr[i].cat.categories = means
    X_test_ctr[i].cat.categories = means

In [48]:
l.fit(X_train_ctr,y_train)
y_pred = l.predict_proba(X_test_ctr)
print(log_loss(y_test,y_pred))
r.fit(X_train_ctr,y_train)
y_pred = r.predict_proba(X_test_ctr)
print(log_loss(y_test,y_pred))

0.42873476553514345
0.41423371514431023


In [50]:
from gensim.models.word2vec import Word2Vec

In [51]:
features

[3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

In [ ]:
from gensim.models.word2vec import Word2Vec
from random import shuffle
size=6
window=8
x_w2v = copy.deepcopy(data.iloc[:,features])
names = list(x_w2v.columns.values)
for i in names:
    x_w2v[i]=x_w2v[i].astype('category')
    x_w2v[i].cat.categories = ["Feature %s %s" % (i,g) for g in x_w2v[i].cat.categories]
x_w2v = x_w2v.values.tolist()
for i in x_w2v:
    shuffle(i)
w2v = Word2Vec(x_w2v,size=size,window=window)
X_train_w2v = copy.copy(X_train)
X_test_w2v = copy.copy(X_test)
for i in names:
    X_train_w2v[i]=X_train_w2v[i].astype('category')
    X_train_w2v[i].cat.categories = ["Feature %s %s" % (i,g) for g in X_train_w2v[i].cat.categories]
for i in names:
    X_test_w2v[i]=X_test_w2v[i].astype('category')
    X_test_w2v[i].cat.categories = ["Feature %s %s" % (i,g) for g in X_test_w2v[i].cat.categories]
X_train_w2v = X_train_w2v.values
X_test_w2v = X_test_w2v.values
x_w2v_train = np.random.random((len(X_train_w2v),size*X_train_w2v.shape[1]))
for j in range(X_train_w2v.shape[1]):
    for i in range(X_train_w2v.shape[0]):
        if X_train_w2v[i,j] in w2v.wv: # .wv is added by 41
            x_w2v_train[i,j*size:(j+1)*size] = w2v.wv[X_train_w2v[i,j]]
x_w2v_test = np.random.random((len(X_test_w2v),size*X_test_w2v.shape[1]))
for j in range(X_test_w2v.shape[1]):
    for i in range(X_test_w2v.shape[0]):
        if X_test_w2v[i,j] in w2v.wv:
            x_w2v_test[i,j*size:(j+1)*size] = w2v.wv[X_test_w2v[i,j]]

In [57]:
l.fit(x_w2v_train,y_train)
y_pred = l.predict_proba(x_w2v_test)
print(log_loss(y_test,y_pred))
r.fit(x_w2v_train,y_train)
y_pred = r.predict_proba(x_w2v_test)
print(log_loss(y_test,y_pred))

0.4170294511369084
0.41016326074048537


# Category embedding with deep learning

In [37]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Reshape
from keras.layers import Merge
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint
import h5py
import copy
X_train_dnn = copy.copy(X_train).values # make it np.ndarray
X_test_dnn = copy.copy(X_test).values

In [38]:
les = []
for i in range(X_train_dnn.shape[1]):
    le = preprocessing.LabelEncoder()
    le.fit(data.iloc[:,features].iloc[:, i])
    les.append(le)
    X_train_dnn[:, i] = le.transform(X_train_dnn[:, i])
    X_test_dnn[:, i] = le.transform(X_test_dnn[:, i])

In [39]:
def split_features(X):
    X_list = []
    
    C1 = X[..., [0]]
    X_list.append(C1)
    
    banner_pos = X[..., [1]]
    X_list.append(banner_pos)
    
    site_id = X[..., [2]]
    X_list.append(site_id)

    site_domain = X[..., [3]]
    X_list.append(site_domain)
    
    site_category = X[..., [4]]
    X_list.append(site_category)
    
    app_id = X[..., [5]]
    X_list.append(app_id)
    
    app_domain = X[..., [6]]
    X_list.append(app_domain)
    
    app_category = X[..., [7]]
    X_list.append(app_category)

    device_id = X[..., [8]]
    X_list.append(device_id)
    
    device_model = X[..., [9]]
    X_list.append(device_model)
    
    device_type = X[..., [10]]
    X_list.append(device_type)

    device_conn_type = X[..., [11]]
    X_list.append(device_conn_type)
    
    C14 = X[..., [12]]
    X_list.append(C14)

    C15 = X[..., [13]]
    X_list.append(C15)
    
    C16 = X[..., [14]]
    X_list.append(C16)
    
    C17 = X[..., [15]]
    X_list.append(C17)
    
    C18 = X[..., [16]]
    X_list.append(C18)
    
    C19 = X[..., [17]]
    X_list.append(C19)

    C20 = X[..., [18]]
    X_list.append(C20)
    
    C21 = X[..., [19]]
    X_list.append(C21)
    
    return X_list

In [41]:
class NN_with_EntityEmbedding(object):
    def __init__(self, X_train, y_train, X_val, y_val):
        self.nb_epoch = 1
        self.__build_keras_model()
        self.fit(X_train, y_train, X_val, y_val)
    def preprocessing(self, X):
        X_list = split_features(X)
        return X_list
    def __build_keras_model(self):
        models = []
        model_C1= Sequential()
        model_C1.add(Embedding(len(les[0].classes_), 3, input_length=1))
        model_C1.add(Reshape(target_shape=(3,)))
        models.append(model_C1)
        
        model_banner_pos = Sequential()
        model_banner_pos.add(Embedding(len(les[1].classes_), 3, input_length=1))
        model_banner_pos.add(Reshape(target_shape=(3,)))
        models.append(model_banner_pos)
        
        model_site_id = Sequential()
        model_site_id.add(Embedding(len(les[2].classes_), 8, input_length=1))
        model_site_id.add(Reshape(target_shape=(8,)))
        models.append(model_site_id)
        
        site_domain = Sequential()
        site_domain.add(Embedding(len(les[3].classes_), 8, input_length=1))
        site_domain.add(Reshape(target_shape=(8,)))
        models.append(site_domain)
        
        site_category = Sequential()
        site_category.add(Embedding(len(les[4].classes_), 3, input_length=1))
        site_category.add(Reshape(target_shape=(3,)))
        models.append(site_category)
        
        app_id = Sequential()
        app_id.add(Embedding(len(les[5].classes_), 8, input_length=1))
        app_id.add(Reshape(target_shape=(8,)))
        models.append(app_id)
        
        app_domain = Sequential()
        app_domain.add(Embedding(len(les[6].classes_), 4, input_length=1))
        app_domain.add(Reshape(target_shape=(4,)))
        models.append(app_domain)
        
        app_category = Sequential()
        app_category.add(Embedding(len(les[7].classes_), 3, input_length=1))
        app_category.add(Reshape(target_shape=(3,)))
        models.append(app_category)
        
        device_id = Sequential()
        device_id.add(Embedding(len(les[8].classes_), 10, input_length=1))
        device_id.add(Reshape(target_shape=(10,)))
        models.append(device_id)
        
        device_model = Sequential()
        device_model.add(Embedding(len(les[9].classes_), 8, input_length=1))
        device_model.add(Reshape(target_shape=(8,)))
        models.append(device_model)
        
        device_type = Sequential()
        device_type.add(Embedding(len(les[10].classes_), 2, input_length=1))
        device_type.add(Reshape(target_shape=(2,)))
        models.append(device_type)
        
        device_conn_type = Sequential()
        device_conn_type.add(Embedding(len(les[11].classes_), 2, input_length=1))
        device_conn_type.add(Reshape(target_shape=(2,)))
        models.append(device_conn_type)
        
        C14 = Sequential()
        C14.add(Embedding(len(les[12].classes_), 8, input_length=1))
        C14.add(Reshape(target_shape=(8,)))
        models.append(C14)
        
        C15 = Sequential()
        C15.add(Embedding(len(les[13].classes_), 3, input_length=1))
        C15.add(Reshape(target_shape=(3,)))
        models.append(C15)
        
        C16 = Sequential()
        C16.add(Embedding(len(les[14].classes_), 3, input_length=1))
        C16.add(Reshape(target_shape=(3,)))
        models.append(C16)
        
        C17 = Sequential()
        C17.add(Embedding(len(les[15].classes_), 4, input_length=1))
        C17.add(Reshape(target_shape=(4,)))
        models.append(C17)
        
        C18 = Sequential()
        C18.add(Embedding(len(les[16].classes_), 2, input_length=1))
        C18.add(Reshape(target_shape=(2,)))
        models.append(C18)
        
        C19 = Sequential()
        C19.add(Embedding(len(les[17].classes_), 4, input_length=1))
        C19.add(Reshape(target_shape=(4,)))
        models.append(C19)
        
        C20 = Sequential()
        C20.add(Embedding(len(les[18].classes_), 5, input_length=1))
        C20.add(Reshape(target_shape=(5,)))
        models.append(C20)
        
        C21 = Sequential()
        C21.add(Embedding(len(les[19].classes_), 4, input_length=1))
        C21.add(Reshape(target_shape=(4,)))
        models.append(C21)
        
        self.model = Sequential()
        self.model.add(Merge(models, mode='concat'))
        self.model.add(Dense(150, kernel_initializer='uniform'))
        self.model.add(Activation('relu'))
        self.model.add(Dense(250, kernel_initializer='uniform'))
        self.model.add(Activation('relu'))
        self.model.add(Dense(1))
        self.model.add(Activation('sigmoid'))
        
        self.model.compile(loss='binary_crossentropy',
                           optimizer='adam',
                           metrics=['acc'])
        
    def fit(self, X_train, y_train, X_val, y_val):
        self.model.fit(self.preprocessing(X_train), y_train,
                       validation_data=(self.preprocessing(X_val), y_val),
                       epochs=self.nb_epoch, batch_size=128,
                       )
        


In [42]:
dnn = NN_with_EntityEmbedding(X_train_dnn, y_train, X_test_dnn, y_test)   
weights = dnn.model.get_weights()

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/ipykernel_launcher.py:112: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Train on 79998 samples, validate on 20002 samples
Epoch 1/1
79998/79998 [==============================] - 11s 140us/step - loss: 0.4229 - acc: 0.8242 - val_loss: 0.4063 - val_acc: 0.8293


In [43]:
dnn.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 95)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 150)               14400     
_________________________________________________________________
activation_1 (Activation)    (None, 150)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 250)               37750     
_________________________________________________________________
activation_2 (Activation)    (None, 250)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 251       
_________________________________________________________________
activation_3 (Activation)    (None, 1)                 0         
Total para

In [44]:
type(weights)

list

In [45]:
len(weights)

46

In [48]:
for i in range(0,40,1):
    if len(weights[i]) != 0:
        print(i, weights[i][0].shape, len(weights[i]))#, weights[i][0])

0 (6, 3) 1
2 (5, 3) 1
4 (893, 8) 1
6 (780, 8) 1
8 (16, 3) 1
10 (704, 8) 1
12 (55, 4) 1
14 (19, 3) 1
16 (7202, 10) 1
18 (2473, 8) 1
20 (4, 2) 1
22 (4, 2) 1
24 (420, 8) 1
26 (5, 3) 1
28 (6, 3) 1
30 (128, 4) 1
32 (4, 2) 1
34 (37, 4) 1
36 (137, 5) 1
38 (29, 4) 1


In [49]:
x_w2v = copy.deepcopy(data.iloc[:,features])
names = list(x_w2v.columns.values)
for i in names:
    x_w2v[i]=x_w2v[i].astype('category')
    x_w2v[i].cat.categories = ["Feature %s %s" % (i,g) for g in x_w2v[i].cat.categories]
    print(len(x_w2v[i].cat.categories), x_w2v[i].cat.categories.values[:min(2,len(x_w2v[i].cat.categories))])

6 ['Feature C1 1001' 'Feature C1 1002']
5 ['Feature banner_pos 0' 'Feature banner_pos 1']
893 ['Feature site_id 00255fb4' 'Feature site_id 006ab81e']
780 ['Feature site_domain 005b495a' 'Feature site_domain 00e1b9c0']
16 ['Feature site_category 0569f928' 'Feature site_category 110ab22d']
704 ['Feature app_id 000d6291' 'Feature app_id 00848fac']
55 ['Feature app_domain 0654b444' 'Feature app_domain 15ec7f39']
19 ['Feature app_category 07d7df22' 'Feature app_category 09481d60']
7202 ['Feature device_id 0003e558' 'Feature device_id 000ef707']
2473 ['Feature device_model 00097428' 'Feature device_model 0009f4d7']
4 ['Feature device_type 0' 'Feature device_type 1']
4 ['Feature device_conn_type 0' 'Feature device_conn_type 2']
420 ['Feature C14 375' 'Feature C14 377']
5 ['Feature C15 120' 'Feature C15 216']
6 ['Feature C16 20' 'Feature C16 36']
128 ['Feature C17 112' 'Feature C17 122']
4 ['Feature C18 0' 'Feature C18 1']
37 ['Feature C19 33' 'Feature C19 35']
137 ['Feature C20 -1' 'Feature C

In [50]:
n = 0
for i in range(0,40,2):
    n+=(weights[i][0].shape)[1]
n

95

In [51]:
X_train_dnn.shape

(79998, 20)

In [52]:
np.unique(X_train_dnn[:,0])

array([0, 1, 2, 3, 4, 5], dtype=object)

In [53]:
x_dnn_train = np.random.random((len(X_train_dnn),n))
start_ind=0
for j in range(X_train_dnn.shape[1]):
    mat = weights[j*2][0]
    dim = mat.shape[1]
    for i in range(X_train_dnn.shape[0]):
        x_dnn_train[i,start_ind:start_ind+dim]=mat[X_train_dnn[i,j]]
    start_ind += dim

x_dnn_test = np.random.random((len(X_test_dnn),n))
start_ind=0
for j in range(X_test_dnn.shape[1]):
    mat = weights[j*2][0]
    dim = mat.shape[1]
    for i in range(x_dnn_test.shape[0]):
        x_dnn_test[i,start_ind:start_ind+dim]=mat[X_test_dnn[i,j]]
    start_ind += dim

In [54]:
l.fit(x_dnn_train,y_train)
y_pred = l.predict_proba(x_dnn_test)
print(log_loss(y_test,y_pred))
r.fit(x_dnn_train,y_train)
y_pred = r.predict_proba(x_dnn_test)
print(log_loss(y_test,y_pred))

0.4120461370111313
0.41186738799763994


# using better keras embedding code

In [6]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload

%autoreload 2

from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs 

['', '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload', '/home/kai/.local/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions', '/home/kai/.ipython', '/home/kai/data/shiyi/AlphaBoosting/automl']


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
X_train_ordinal_df = pd.DataFrame(X_train_ordinal)

In [8]:
X_test_ordinal_df = pd.DataFrame(X_test_ordinal)

In [16]:
embed_outdim = [3, 3, 8, 8, 3, 8, 4, 3, 10, 8, 2, 2, 8, 3, 3, 4, 2, 4, 5, 4]
cat_cols = X_train_ordinal_df.columns.values # this df is from np array, so its column names are integer
embed_outdim_dict = dict(zip(cat_cols, embed_outdim))

params = {
    'nn_seed': 2018, 
    'ep_for_lr': 1,
    'lr_init': 0.01,
    'lr_fin': 0.01,
    'batch_size': 128,
    "pred_batch_size": 100000,
    'max_ep': 1,
    'patience': 1,
    'cat_emb_outdim': embed_outdim_dict,
    'dense_units': 150,
    'num_dense_n_layers': 1,
    'drop_rate': 0,
    'combined_dense_n_layers': 2
}

In [17]:
model, tr_dict, va_dict, te_dict = nn_libs.get_model(params, X_train=X_train_ordinal_df, X_val=X_test_ordinal_df, X_test=X_test_ordinal_df.head(5), categorical_features=X_train_ordinal_df.columns.values)

In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
0 (InputLayer)                  (None, 1)            0                                            
__________________________________________________________________________________________________
1 (InputLayer)                  (None, 1)            0                                            
__________________________________________________________________________________________________
2 (InputLayer)                  (None, 1)            0                                            
__________________________________________________________________________________________________
3 (InputLayer)                  (None, 1)            0                                            
__________________________________________________________________________________________________
4 (InputLa

In [21]:
model.fit(tr_dict, y_train, validation_data=[va_dict, y_test], epochs=params['max_ep'], batch_size=128, verbose=1)

Train on 79998 samples, validate on 20002 samples
Epoch 1/1
79998/79998 [==============================] - 11s 137us/step - loss: 0.4074 - acc: 0.8267 - val_loss: 0.4071 - val_acc: 0.8276


In [23]:
layer_list = []
for layer in model.get_config()['layers']:
    layer_list.append(layer['name'])

cat_cols_to_weight_idx = dict(zip(cat_cols, [layer_list.index(str(col)) for col in cat_cols]))

In [24]:
wei = model.get_weights()

In [25]:
type(wei), len(wei)

(list, 26)

In [26]:
for i in range(len(wei)):
    if len(wei[i]) != 0:
        print('index in weight {:3} | embedding dimension {:7}'.format(i, str(wei[i].shape)))

index in weight   0 | embedding dimension (6, 3) 
index in weight   1 | embedding dimension (5, 3) 
index in weight   2 | embedding dimension (893, 8)
index in weight   3 | embedding dimension (780, 8)
index in weight   4 | embedding dimension (16, 3)
index in weight   5 | embedding dimension (704, 8)
index in weight   6 | embedding dimension (55, 4)
index in weight   7 | embedding dimension (19, 3)
index in weight   8 | embedding dimension (7202, 10)
index in weight   9 | embedding dimension (2473, 8)
index in weight  10 | embedding dimension (4, 2) 
index in weight  11 | embedding dimension (4, 2) 
index in weight  12 | embedding dimension (420, 8)
index in weight  13 | embedding dimension (5, 3) 
index in weight  14 | embedding dimension (6, 3) 
index in weight  15 | embedding dimension (128, 4)
index in weight  16 | embedding dimension (4, 2) 
index in weight  17 | embedding dimension (37, 4)
index in weight  18 | embedding dimension (137, 5)
index in weight  19 | embedding dimensi

In [27]:
embedded_df_num_cols = 0
for col, idx in cat_cols_to_weight_idx.items():
    embedded_df_num_cols +=(wei[idx][0].shape)[0]
embedded_df_num_cols

95

In [156]:
X_train_dnn.shape

(80000, 20)

In [35]:
x_dnn_train = np.random.random((len(X_train_ordinal),embedded_df_num_cols))
start_ind=0
for col, wei_idx in cat_cols_to_weight_idx.items():
    mat = wei[wei_idx]
    dim = mat.shape[1]
    for i in range(X_train_ordinal.shape[0]):
        x_dnn_train[i,start_ind:start_ind+dim]=mat[X_train_ordinal[i,int(col)]]
    start_ind += dim

x_dnn_test = np.random.random((len(X_test_ordinal),embedded_df_num_cols))
start_ind=0
for col, wei_idx in cat_cols_to_weight_idx.items():
    mat = wei[wei_idx]
    dim = mat.shape[1]
    for i in range(X_test_ordinal.shape[0]):
        x_dnn_test[i,start_ind:start_ind+dim]=mat[X_test_ordinal[i,int(col)]]
    start_ind += dim

In [36]:
l.fit(x_dnn_train,y_train)
y_pred = l.predict_proba(x_dnn_test)
print(log_loss(y_test,y_pred))
r.fit(x_dnn_train,y_train)
y_pred = r.predict_proba(x_dnn_test)
print(log_loss(y_test,y_pred))

0.4204629456026463
0.4164564692446847
